In [ ]:
# %pip install pyfolio
# %pip install pandas
# %pip install numpy
# %pip install datetime
import warnings
warnings.filterwarnings('ignore')
import pyfolio as pf
import pandas as pd
import numpy as np
from leveraged_meta_lib import leveraged_meta
import common_perf_ana as cpa
import datetime

# %pip uninstall yfinance // we have to uninstall the old version and install the new (2023-01) version
# %pip install yfinance
# %pip install nbformat
# %pip install plotly
# %pip install pandas-market-calendars

In [ ]:

meta_parameters = {
    'meta_rebalance_unit' : 'Month',
    'meta_rebalance_freq' : 1,
    'meta_rebalance_shift' : 0,
    'meta_lb_period' : 3,
    'meta_skipped_period' : 0,
    'meta_fixed_substrat_weights' : {'DualMom' : 2, 'BAA_AggDef' : 2, 'BAA_BalDef' : 2, 'TAA' : 2, 'KellerProtMom' : 2, 'NovellTactBond' : 1, 'HAA' : 1},
    'meta_rel_mom_substrat_weights' : [1, 2, 3, 4, 5, 6, 7], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf., Fifth Perf., Sixth Perf., Seventh Perf.]
    'meta_sharpe_substrat_weights' : [1, 2, 3, 4, 5, 6, 7], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf., Fifth Perf., Sixth Perf., Seventh Perf.]
    'meta_sortino_substrat_weights' : [1, 2, 3, 4, 5, 6, 7], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf., Fifth Perf., Sixth Perf., Seventh Perf.]
    'meta_rel_mom_abs_threshold' : -1,
    'meta_sharpe_abs_threshold' : -100,
    'meta_sortino_abs_threshold' : -100,
    'meta_start_date' : '2004-01-01',
    'meta_end_date' : datetime.date.today(),
    'used_substrat_weights' : 'fixed_based' # 'fixed_based', 'ew_based', 'rel_mom_based', 'sharpe_based', 'sortino_based' 
}

meta_leverage_parameters = {
    # 'meta_overall_leverage' : 1,
    # 'meta_ETF_perf_leverage_threshold' : [0.2, 0.4, 0.6, 0.8, 1],
    # 'meta_ETF_perf_leverage' : [1, 1, 1, 1, 1],
    # 'meta_monthly_seas_based_leverage' : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # 'meta_leverage_lookback_years': 5,
    # 'meta_tlt_based_leverage' : [1, 1, 1, 1] #[Bull, Rebound, Correction, Bear]
    # 'meta_overall_leverage' : 1,
    # 'meta_ETF_perf_leverage_threshold' : [0.2, 0.4, 0.6, 0.8, 1],
    # 'meta_ETF_perf_leverage' : [2, 1.5, 1, 0.75, 0.5],
    # 'meta_monthly_seas_based_leverage' : [1.5, 1.5, 1.25, 1.25, 1, 1, 1, 1, 1, 1, 1, 1],
    # 'meta_leverage_lookback_years': 5,
    # 'meta_tlt_based_leverage' : [1.1, 1.1, 1, 0.8] #[Bull, Rebound, Correction, Bear]
    'meta_overall_leverage' : 1, # 2023-11-29: Decrease discretionary leverage to 100%
    'meta_ETF_perf_leverage_threshold' : [0.2, 0.4, 0.6, 0.8, 1],
    'meta_ETF_perf_leverage' : [1.5, 1.25, 1, 0.85, 0.7],
    # 'meta_ETF_perf_leverage' : [1, 1, 1, 1, 1],
    'meta_monthly_seas_based_leverage' : [1.5, 1.5, 1.25, 1.25, 1, 1, 1, 1, 1, 1, 0.85, 0.85],
    # 'meta_monthly_seas_based_leverage' : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'meta_leverage_lookback_years': 5,
    'meta_tlt_based_leverage' : [1.25, 1.4, 1, 0.75], #[Bull, Rebound, Correction, Bear],
    'meta_tip_based_leverage': {'AGG': [1, 1, 1, 1, 1], 'BIL': [1, 1, 1, 1, 1], 'BNDX': [1, 1, 1, 1, 1], 'DBC': [0.5, 1, 1.5, 1, 1], 'EEM': [0.5, 1, 1.5, 1, 1], 'EFA': [0.5, 1, 1.5, 1, 1], 'EMB': [1, 1, 1, 1, 1], 'EWJ': [1, 1, 1, 1, 1],
                                'GLD': [1, 1, 1, 1, 1], 'HYG': [1, 1, 1, 1, 1], 'IEF': [1, 1, 1, 1, 1], 'IWM': [0.5, 1, 1.5, 1, 1], 'LQD': [1, 1, 1, 1, 1], 'QQQ': [1, 1.25, 1.5, 1.25, 1], 'SHY': [1, 1, 1, 1, 1], 'SPY': [1, 1.25, 1.5, 1.25, 1],
                                'TIP': [1, 1, 1, 1, 1], 'TLT': [1, 1, 1, 1, 1], 'VEA': [0.5, 1, 1.5, 1, 1], 'VGK': [0.5, 1, 1.5, 1, 1], 'VNQ': [0.5, 1, 1.5, 1, 1], 'VWO': [0.5, 1, 1.5, 1, 1]}
    # 'meta_tip_based_leverage': {'AGG': [1, 1, 1, 1, 1], 'BIL': [1, 1, 1, 1, 1], 'BNDX': [1, 1, 1, 1, 1], 'DBC': [1, 1, 1, 1, 1], 'EEM': [1, 1, 1, 1, 1], 'EFA': [1, 1, 1, 1, 1], 'EMB': [1, 1, 1, 1, 1], 'EWJ': [1, 1, 1, 1, 1],
    #                            'GLD': [1, 1, 1, 1, 1], 'HYG': [1, 1, 1, 1, 1], 'IEF': [1, 1, 1, 1, 1], 'IWM': [1, 1, 1, 1, 1], 'LQD': [1, 1, 1, 1, 1], 'QQQ': [1, 1, 1, 1, 1], 'SHY': [1, 1, 1, 1, 1], 'SPY': [1, 1, 1, 1, 1],
    #                            'TIP': [1, 1, 1, 1, 1], 'TLT': [1, 1, 1, 1, 1], 'VEA': [1, 1, 1, 1, 1], 'VGK': [1, 1, 1, 1, 1], 'VNQ': [1, 1, 1, 1, 1], 'VWO': [1, 1, 1, 1, 1]}
}

taa_parameters = {
    'taa_ticker_list' : ['VNQ', 'EEM', 'DBC', 'SPY', 'QQQ', 'TLT'],
    'taa_perc_ch_lb_list' : [60, 120, 180, 250],
    'taa_vol_lb' : 20,
    'taa_perc_ch_up_thres' : 0.75,
    'taa_perc_ch_low_thres' : 0.25,
    'taa_rebalance_unit' : 'Month',
    'taa_rebalance_freq' : 1,
    'taa_rebalance_shift' : 0
}

bold_parameters = {
    'baa_ticker_list_canary' : ['SPY', 'EEM', 'EFA', 'AGG'],
    'baa_ticker_list_defensive' : ['TIP', 'DBC', 'BIL', 'IEF', 'TLT', 'LQD', 'AGG'],
    # 'baa_ticker_list_aggressive' : ['QQQ'],
    'baa_ticker_list_aggressive' : ['QQQ', 'EEM', 'EFA', 'AGG'],
    'baa_ticker_list_balanced' : ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'VNQ', 'DBC', 'GLD', 'TLT', 'HYG', 'LQD'],
    'baa_rebalance_unit' : 'Month',
    'baa_rebalance_freq' : 1,
    'baa_rebalance_shift' : 0,
    'baa_skipped_period' : 0,
    'baa_no_played_ETFs' : {'offAgg' : 1, 'offBal' : 6, 'def' : 3},
    'baa_abs_threshold' : -1
}

dual_mom_parameters = {
    'dm_tickers_list' : ['VNQ', 'EEM', 'DBC', 'SPY', 'TLT', 'SHY'],
    'dm_rebalance_unit' : 'Month',
    'dm_rebalance_freq' : 1,
    'dm_rebalance_shift' : 0,
    'dm_lb_period' : 4,
    'dm_skipped_period' : 0,
    'dm_no_played_ETFs' : 3,
    'dm_sub_rank_weights' : {'relMom' : 0.5, 'volatility' : 0.25, 'correlation' : 0.25},
    'dm_abs_threshold' : 0
}

keller_protmom_parameters = {
    'protmom_tickers_list' : ['SPY', 'QQQ', 'IWM', 'VNQ', 'DBC', 'GLD', 'HYG', 'LQD', 'TLT'],
    'protmom_rebalance_unit' : 'Month',
    'protmom_rebalance_freq' : 1,
    'protmom_rebalance_shift' : 0,
    'protmom_correl_lb_months' : 12,
    'protmom_lb_periods' : [1, 3, 6, 12],
    'protmom_lb_weights' : [1, 1, 1, 1],
    'protmom_selected_ETFs' : 3
}

novell_tactbond_parameters = {
    'tactbond_tickers_list' : ['BNDX', 'EMB', 'HYG', 'IEF', 'LQD', 'SHY', 'TIP', 'TLT'],
    'tactbond_rebalance_unit' : 'Month',
    'tactbond_rebalance_freq' : 1,
    'tactbond_rebalance_shift' : 0,
    'tactbond_absolute_threshold' : 0,
    'tactbond_threshold_type' : 1, # 1 - Relative threshold to BIL, 0 - Absolute threshold
    'tactbond_cash_subs' : 1, # 1 - BIL, 0 - Cash
    'tactbond_lb_periods' : [1, 3, 6, 12],
    'tactbond_lb_weights' : [0, 0, 1, 0],
    'tactbond_selected_ETFs' : 3
}

hybrid_aa_parameters = {
    'haa_ticker_list_canary' : ['TIP'],
    'haa_ticker_list_defensive' : ['BIL', 'IEF'],
    'haa_ticker_list_offensive' : ['SPY', 'IWM', 'VWO', 'VEA', 'VNQ', 'DBC', 'IEF', 'TLT'],
    'haa_rebalance_unit' : 'Month',
    'haa_rebalance_freq' : 1,
    'haa_rebalance_shift' : 0,
    'haa_skipped_period' : 0,
    'haa_no_played_ETFs' : {'off' : 4, 'def' : 1},
    'haa_abs_threshold' : 0
}

pv_played_fin, strat_played_rets, strat_played_weights, pos_played_fin, cash_played_fin, strat_played_curr_weights, strat_unlev_curr_weights, ETF_curr_levs, monthly_curr_lev, tlt_curr_lev, curr_tip_quint = leveraged_meta(meta_parameters, taa_parameters, bold_parameters, dual_mom_parameters, keller_protmom_parameters, novell_tactbond_parameters, meta_leverage_parameters, hybrid_aa_parameters)
tip_quintNames = ['Lowest (0% - 20%)', 'Second (20% - 40%)', 'Median (40% - 60%)', 'Fourth (60% - 80%)', 'Highest (80% - 100%)']
pd.set_option('display.float_format', '{:.2f}%'.format)
print('Unleveraged weights: ')
display(strat_unlev_curr_weights * 100)
print('Leveraged weights: ')
display(strat_played_curr_weights * 100)
print('Used overall leverage: ', '{:.2f}%'.format(meta_leverage_parameters['meta_overall_leverage'] * 100))
print('Used TLT based leverage: ', '{:.2f}%'.format(tlt_curr_lev * 100))
print('Used monthly performance based leverage: ', '{:.2f}%'.format(monthly_curr_lev * 100))
print('Current TIP quintile: ', tip_quintNames[int(curr_tip_quint) - 1])
print('Used ETF performance based leverage: ')
display(ETF_curr_levs * 100)
cpa.performance_indicators(pv_played_fin)
